# Metadata

**L1 Taxonomy** - Backend Integration

**L2 Taxonomy** - Webhooks

**Subtopic** - Logging and auditing all received webhook events for debugging

**Use Case** - Develop a Python script that listens for incoming webhook events, logs the event details, and stores them in a local JSON file for auditing and debugging purposes. This script should handle HTTP POST requests, extract relevant data from the request body, and append it to a JSON file in a structured format. This process will allow for easy review and debugging of all received webhook events.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt



Problem Description
Create a Python 3.10+ module named `webhook_event_logger`. It must run a small HTTP service that listens only on the path `/webhook`. Each valid POST request must be

1. written as one JSON line to `events.jsonl`,
2. audited by appending a SHA‑256 hash‑chain entry to `audit.log`, and
3. acknowledged with a JSON confirmation containing a deterministic `event_id`.

The module must also supply a command‑line interface with two sub‑commands:
• `serve` – start the HTTP listener.
• `verify` – check the audit log for tampering and exit 0 if intact, 1 otherwise.

Everything must rely on the Python standard library only.

Input Format and Constraints
HTTP layer

* Method: POST (others-> 405)
* Path: `/webhook` (others-> 404)
* Content‑Type header: `application/json` (others-> 415)
* Content‑Length header: present integer ≤ 1 MiB (missing-> 411, too large-> 413)
* Body: UTF‑8 JSON object, no raw `\r` or `\n` bytes (invalid UTF‑8-> 400, control chars-> 400)

Payload schema

```
{
  "event": non‑empty string, ≤ 255 UTF‑8 bytes, no ASCII control characters,
  "data":  any JSON value, optional
}
```

CLI invocation

```
python -m webhook_event_logger serve  --root <dir> [--host 0.0.0.0] [--port 8000]
python -m webhook_event_logger verify --root <dir>
```

Module API used by the grader (see signature block below).

Expected Output Format
Success: HTTP 200, body `{"status":"ok","id":"<event_id>"}`.
Error: JSON body `{"error":"<message>"}` with the status listed above.
`events.jsonl`: each line is `{"ts":"<iso‑8601 UTC>","payload":{...}}`.
`audit.log`: each line is `<b64url(chain_hash)> <event_id>\n`, where
`chain_hash = sha256(prev_event_id || event_json)` and `prev_event_id = "0"*64` for the first event.

Examples

```bash
python -m webhook_event_logger serve --root ./data --port 9000 &
curl -X POST http://localhost:9000/webhook \
     -H "Content-Type: application/json" \
     -d '{"event":"order.created","data":{"id":42}}'
python -m webhook_event_logger verify --root ./data   # prints “Audit log OK”
```

```py
from webhook_event_logger import EventLogger
elog = EventLogger("./sandbox")
eid  = elog.append({"event":"ping"})
assert elog.verify() is True
```


# Requirements


REQUIREMENTS
Explicit and Implicit Points
• Reject all malformed HTTP requests with the exact status codes specified above.
• Generate `event_id` as the hex SHA‑256 of the stored event JSON.
• Use a `threading.Lock` plus temp‑file + `os.replace` for crash‑safe writes.
• Auto‑rotate `events.jsonl` to `events‑YYYYMMDD‑HHMMSS.jsonl` when its size exceeds 10 MiB.
• `verify()` must scan the audit log linearly and return False, not raise, on any break or missing event.

Solution Expectations
• Handle at least 500 concurrent POST requests without race conditions or data loss.
• Verify a 50 000‑line audit file in under one second on a modern CPU.
• Graceful shutdown on SIGINT/SIGTERM without leaving partial lines.

Signatures of Expected Functions

```py
class EventLogger:
    def __init__(self, root_dir: str) -> None: ...
    def append(self, payload: Mapping[str, Any]) -> str: ...   # returns event_id
    def verify(self) -> bool: ...
```

Edge‑Case Behaviour

* Body larger than 1 MiB-> 413
* Missing Content‑Length-> 411
* Invalid UTF‑8-> 400
* `event` absent, empty, too long, or containing control chars-> 422
* JSON root not an object-> 400
* Tampered audit or events file-> `verify()` returns False
* Missing audit file-> `verify()` returns True

Constraints
• Standard library only (no third‑party imports, no `asyncio`, no subprocess).
• No quadratic algorithms.
• No forbidden built‑ins such as `eval`, `exec`, `os.system`.
• No sleep‑based timing loops.

Important Notes
Validation is required. On any constraint violation:

* HTTP route must return the designated error code and JSON body.
* `EventLogger.append()` must raise ValueError for schema errors or TypeError for non‑mapping payloads.
* `EventLogger.verify()` must never raise; it returns False on corruption.


In [ ]:
# code

"""
webhook_event_logger.py

A hardened, standard-library-only webhook event logger with tamper-evident
audit chaining and robust edge-case handling.

Features
--------
- Listens on POST /webhook for JSON payloads.
- Enforces Content-Type, Content-Length, UTF-8, and schema constraints.
- Appends each event to events.jsonl and audit.log with SHA-256 hash chain.
- Atomic, thread-safe writes via tempfile + os.replace and threading.Lock.
- Auto-rotates events.jsonl when it exceeds 10 MiB.
- Provides CLI:
    * serve  — start HTTP listener
    * verify — check audit log integrity (exit 0 if OK, 1 if broken)
"""

import argparse
import base64
import hashlib
import http.server
import json
import os
import signal
import sys
import tempfile
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Mapping

# Constants
_MAX_BODY_BYTES = 1_048_576  # 1 MiB
_MAX_EVENTS_FILE_BYTES = 10 * _MAX_BODY_BYTES
_EOL = "\n"
_LOCK = threading.Lock()


def _b64url(data: bytes) -> str:
    return base64.urlsafe_b64encode(data).rstrip(b"=").decode("ascii")


def _atomic_append(path: Path, text: str) -> None:
    fd, tmp_path = tempfile.mkstemp(dir=path.parent)
    try:
        with os.fdopen(fd, "w", encoding="utf-8", newline="") as tmp:
            if path.exists():
                tmp.write(path.read_text("utf-8"))
            if not text.endswith(_EOL):
                text += _EOL
            tmp.write(text)
        os.replace(tmp_path, path)
    finally:
        if os.path.exists(tmp_path):
            os.unlink(tmp_path)


def _read_last_line(path: Path) -> str | None:
    if not path.exists():
        return None
    with path.open("rb") as f:
        try:
            f.seek(-2, os.SEEK_END)
            while f.read(1) != b"\n":
                f.seek(-2, os.SEEK_CUR)
        except OSError:
            f.seek(0)
        return f.readline().decode("utf-8", "replace").rstrip(_EOL) or None


def _lookup_event_json(events_path: Path, event_id: str) -> str:
    if not events_path.exists():
        raise FileNotFoundError("events file missing")
    with events_path.open("r", encoding="utf-8") as f:
        for line in f:
            candidate = line.rstrip(_EOL)
            if hashlib.sha256(candidate.encode()).hexdigest() == event_id:
                return candidate
    raise ValueError(f"event_id {event_id!r} not found")


class EventLogger:
    def __init__(self, root_dir: str | os.PathLike[str]) -> None:
        self.root = Path(root_dir).resolve()
        self.root.mkdir(parents=True, exist_ok=True)
        self.events_file = self.root / "events.jsonl"
        self.audit_file = self.root / "audit.log"

    def append(self, payload: Mapping[str, Any]) -> str:
        if not isinstance(payload, Mapping):
            raise TypeError("payload must be a mapping type")
        event = payload.get("event")
        if not isinstance(event, str) or not event.strip():
            raise ValueError("'event' must be a non-empty string")
        if len(event.encode("utf-8")) > 255:
            raise ValueError("'event' exceeds 255 bytes")
        if any(ord(ch) < 32 for ch in event):
            raise ValueError("'event' contains control characters")

        timestamp = datetime.now(timezone.utc).isoformat()
        record = {"ts": timestamp, "payload": dict(payload)}
        record_json = json.dumps(record, ensure_ascii=False)

        event_id = hashlib.sha256(record_json.encode()).hexdigest()
        prev_line = _read_last_line(self.audit_file)
        prev_id = prev_line.split(" ")[1] if prev_line else "0" * 64
        chain_hash = hashlib.sha256((prev_id + record_json).encode()).digest()
        audit_line = f"{_b64url(chain_hash)} {event_id}"

        with _LOCK:
            _atomic_append(self.events_file, record_json)
            _atomic_append(self.audit_file, audit_line)
            self._rotate_events_if_needed()

        return event_id

    def verify(self) -> bool:
        if not self.audit_file.exists():
            return True
        prev_id = "0" * 64
        for line in self.audit_file.read_text("utf-8").splitlines():
            parts = line.split(" ")
            if len(parts) != 2:
                return False
            chain_b64, event_id = parts
            try:
                ev_json = _lookup_event_json(self.events_file, event_id)
            except (FileNotFoundError, ValueError):
                return False
            expected = hashlib.sha256((prev_id + ev_json).encode()).digest()
            if chain_b64 != _b64url(expected):
                return False
            prev_id = event_id
        return True

    def _rotate_events_if_needed(self) -> None:
        if self.events_file.stat().st_size <= _MAX_EVENTS_FILE_BYTES:
            return
        ts = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
        rotated = self.events_file.with_name(f"events-{ts}.jsonl")
        os.replace(self.events_file, rotated)
        self.events_file.touch()


class WebhookHandler(http.server.BaseHTTPRequestHandler):
    logger: EventLogger  # injected by server factory

    def log_message(self, *args: Any) -> None:
        return  # silence

    def _respond(self, code: int, body: dict[str, Any]) -> None:
        payload = json.dumps(body).encode("utf-8")
        self.send_response(code)
        self.send_header("Content-Type", "application/json")
        self.send_header("Content-Length", str(len(payload)))
        self.end_headers()
        self.wfile.write(payload)

    def do_POST(self) -> None:
        if self.path != "/webhook":
            return self._respond(404, {"error": "Not Found"})

        ct = self.headers.get("Content-Type", "").split(";", 1)[0]
        if ct != "application/json":
            return self._respond(415, {"error": "Content-Type must be application/json"})

        try:
            length = int(self.headers.get("Content-Length", ""))
        except ValueError:
            return self._respond(411, {"error": "Content-Length required"})
        if length > _MAX_BODY_BYTES:
            # read and discard to avoid broken pipe
            _ = self.rfile.read(length)
            return self._respond(413, {"error": "Payload too large"})

        raw = self.rfile.read(length)
        try:
            text = raw.decode("utf-8")
        except UnicodeDecodeError:
            return self._respond(400, {"error": "Body must be valid UTF-8"})
        if "\r" in text or "\n" in text:
            return self._respond(400, {"error": "Body contains forbidden control characters"})

        try:
            obj = json.loads(text)
        except json.JSONDecodeError:
            return self._respond(400, {"error": "Invalid JSON"})
        if not isinstance(obj, dict):
            return self._respond(400, {"error": "JSON must be an object"})

        # missing-event → 404
        if "event" not in obj:
            return self._respond(404, {"error": "Not Found"})

        try:
            event_id = self.logger.append(obj)
        except (TypeError, ValueError) as e:
            return self._respond(422, {"error": str(e)})
        except Exception as e:
            return self._respond(500, {"error": f"Internal error: {e}"})

        self._respond(200, {"status": "ok", "id": event_id})


def _serve(root: str, host: str, port: int) -> None:
    logger = EventLogger(root)

    class ThreadedHTTPServer(http.server.ThreadingHTTPServer):
        def finish_request(self, request, client_address):
            handler = WebhookHandler(request, client_address, self)
            handler.logger = logger

    server = ThreadedHTTPServer((host, port), WebhookHandler)

    def _shutdown(signum, frame):
        server.shutdown()

    for sig in (signal.SIGINT, signal.SIGTERM):
        signal.signal(sig, _shutdown)

    print(f"Webhook logger listening on {host}:{port}", file=sys.stderr)
    server.serve_forever()


def _verify(root: str) -> None:
    ok = EventLogger(root).verify()
    sys.exit(0 if ok else 1)


def main() -> None:
    parser = argparse.ArgumentParser(prog="webhook_event_logger")
    sub = parser.add_subparsers(dest="cmd", required=True)

    serve_p = sub.add_parser("serve", help="Run HTTP webhook listener")
    serve_p.add_argument("--root", required=True, help="Root directory for logs")
    serve_p.add_argument("--host", default="0.0.0.0", help="Host to bind")
    serve_p.add_argument("--port", type=int, default=8000, help="Port to bind")

    verify_p = sub.add_parser("verify", help="Verify audit log integrity")
    verify_p.add_argument("--root", required=True, help="Root directory for logs")

    args = parser.parse_args()
    if args.cmd == "serve":
        _serve(args.root, args.host, args.port)
    else:
        _verify(args.root)


if __name__ == "__main__":
    main()


usage: webhook_event_logger [-h] {serve,verify} ...
webhook_event_logger: error: argument cmd: invalid choice: '/root/.local/share/jupyter/runtime/kernel-c0f76e0b-1397-400b-8f4e-a1cefda7f12e.json' (choose from 'serve', 'verify')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.11/argparse.py", line 1919, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/argparse.py", line 2143, in _parse_known_args
    stop_index = consume_positionals(start_index)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/argparse.py", line 2099, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.11/argparse.py", line 1979, in take_action
    argument_values = self._get_values(action, argument_strings)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/argparse.py", line 2528, in _get_values
    self._check_value(action, value[0])
  File "/usr/lib/python3.11/argparse.py", line 2575, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument cmd: invalid choice: '/root/.local/s

TypeError: object of type 'NoneType' has no len()

In [ ]:
# tests

# test.py

import unittest
import os
import json
import tempfile
import shutil
import threading
import http.client
import time
import socket
import sys

import main  # assumes your implementation is in main.py

# Helper to find an available port
def find_free_port():
    s = socket.socket()
    s.bind(('localhost', 0))
    port = s.getsockname()[1]
    s.close()
    return port

class TestEventLogger(unittest.TestCase):
    def setUp(self):
        self.tmpdir = tempfile.mkdtemp()
        self.logger = main.EventLogger(self.tmpdir)

    def tearDown(self):
        shutil.rmtree(self.tmpdir)

    def test_append_and_verify_success(self):
        payload = {"event": "user.created", "data": {"id": 1}}
        eid = self.logger.append(payload)

        # Check events.jsonl
        ev_file = os.path.join(self.tmpdir, 'events.jsonl')
        with open(ev_file, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines()
        self.assertEqual(len(lines), 1)
        rec = json.loads(lines[0])
        self.assertIn('ts', rec)
        self.assertEqual(rec['payload'], payload)

        # Check audit.log
        au_file = os.path.join(self.tmpdir, 'audit.log')
        with open(au_file, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines()
        self.assertEqual(len(lines), 1)
        chain, logged_eid = lines[0].split(' ')
        self.assertEqual(logged_eid, eid)

        # verify should be True
        self.assertTrue(self.logger.verify())

    def test_tamper_audit_detected(self):
        eid = self.logger.append({"event": "test"})
        au_file = os.path.join(self.tmpdir, 'audit.log')
        with open(au_file, 'r+', encoding='utf-8') as f:
            data = f.read()
            f.seek(0)
            f.write('X' + data[1:])
            f.truncate()
        self.assertFalse(self.logger.verify())

    def test_invalid_event_payloads(self):
        with self.assertRaises(TypeError):
            self.logger.append(123)
        with self.assertRaises(ValueError):
            self.logger.append({})            # missing event
        with self.assertRaises(ValueError):
            self.logger.append({'event': ''})
        long_event = 'a' * 256
        with self.assertRaises(ValueError):
            self.logger.append({'event': long_event})

class TestHTTPServer(unittest.TestCase):
    def setUp(self):
        self.tmpdir = tempfile.mkdtemp()
        self.port = find_free_port()
        server = http.server.ThreadingHTTPServer(('localhost', self.port), main.WebhookHandler)
        main.WebhookHandler.logger = main.EventLogger(self.tmpdir)
        self.server = server
        self.thread = threading.Thread(target=server.serve_forever)
        self.thread.daemon = True
        self.thread.start()
        time.sleep(0.1)

    def tearDown(self):
        self.server.shutdown()
        self.thread.join()
        shutil.rmtree(self.tmpdir)

    def http_post(self, body, headers):
        conn = http.client.HTTPConnection('localhost', self.port)
        conn.request('POST', '/webhook', body, headers)
        resp = conn.getresponse()
        data = resp.read().decode('utf-8')
        conn.close()
        return resp.status, data

    def test_http_success(self):
        body = json.dumps({'event': 'ping', 'data': {}})
        status, data = self.http_post(
            body,
            {'Content-Type': 'application/json', 'Content-Length': str(len(body))}
        )
        self.assertEqual(status, 200)
        obj = json.loads(data)
        self.assertIn('id', obj)
        self.assertTrue(main.WebhookHandler.logger.verify())

    def test_http_not_found(self):
        status, _ = self.http_post(
            '{}',
            {'Content-Type': 'application/json', 'Content-Length': '2'}
        )
        self.assertEqual(status, 404)

    def test_http_bad_content_type(self):
        body = json.dumps({'event': 'ping'})
        status, data = self.http_post(
            body,
            {'Content-Type': 'text/plain', 'Content-Length': str(len(body))}
        )
        self.assertEqual(status, 415)
        self.assertIn('error', json.loads(data))

    def test_http_payload_too_large(self):
        body = 'x' * (main._MAX_BODY_BYTES + 1)
        status, _ = self.http_post(
            body,
            {'Content-Type': 'application/json', 'Content-Length': str(len(body))}
        )
        self.assertEqual(status, 413)

    def test_http_invalid_json(self):
        status, _ = self.http_post(
            '{not json}',
            {'Content-Type': 'application/json', 'Content-Length': '10'}
        )
        self.assertEqual(status, 400)

class TestCLI(unittest.TestCase):
    def setUp(self):
        self.tmpdir = tempfile.mkdtemp()

    def tearDown(self):
        shutil.rmtree(self.tmpdir)

    def test_verify_exit_code(self):
        # no events-> exit 0
        with self.assertRaises(SystemExit) as cm:
            main._verify(self.tmpdir)
        self.assertEqual(cm.exception.code, 0)

        # tamper-> exit non-zero
        logger = main.EventLogger(self.tmpdir)
        eid = logger.append({'event': 'e'})
        audit = os.path.join(self.tmpdir, 'audit.log')
        with open(audit, 'r+', encoding='utf-8') as f:
            data = f.read(); f.seek(0); f.write('X'+data[1:]); f.truncate()
        with self.assertRaises(SystemExit) as cm2:
            main._verify(self.tmpdir)
        self.assertNotEqual(cm2.exception.code, 0)

if __name__ == '__main__':
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/c78c742d-c77d-4748-871a-79050726b430?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F82818%2Fview

#### Model code:

```python
import os
import json
import hashlib
import threading
import tempfile
import base64  # <Issue>: The original code forgot to import base64 even though it uses it later.
from datetime import datetime
from http.server import BaseHTTPRequestHandler, HTTPServer
from typing import Any, Mapping

MAX_CONTENT_LENGTH = 1 * 1024 * 1024  # 1 MiB
MAX_EVENT_LENGTH = 255  # Max length for event field
MAX_EVENTS_FILE_SIZE = 10 * 1024 * 1024  # 10 MiB


class EventLogger:
    def __init__(self, root_dir: str) -> None:
        self.root_dir = root_dir
        self.lock = threading.Lock()
        self.events_file = os.path.join(self.root_dir, 'events.jsonl')
        self.audit_file = os.path.join(self.root_dir, 'audit.log')
        os.makedirs(self.root_dir, exist_ok=True)

    def compute_event_id(self, event: dict) -> str:
        """Generate a deterministic event_id from the event JSON."""
        event_json = json.dumps(event, ensure_ascii=False, separators=(',', ':')).encode('utf-8')
        return hashlib.sha256(event_json).hexdigest()

    def append(self, payload: Mapping[str, Any]) -> str:
        """Append a valid event payload to the log files."""
        if not isinstance(payload, dict):
            raise TypeError("Payload must be a mapping object")

        if "event" not in payload or not payload["event"] or len(payload["event"].encode("utf-8")) > MAX_EVENT_LENGTH:
            raise ValueError("Invalid 'event' field")

        if any(ord(c) < 32 and c not in (' ', '\t') for c in payload['event']):  # <Issue>: Tab (0x09) is still a control char and should be rejected.
            raise ValueError("Event field contains control characters")

        with self.lock:
            timestamp = datetime.utcnow().isoformat(timespec='seconds') + 'Z'
            record = {'ts': timestamp, 'payload': payload}
            event_id = self.compute_event_id(record)

            # Write to event file
            record_json = json.dumps(record, ensure_ascii=False, separators=(',', ':'))
            self._write_atomic(self.events_file, record_json + "\n")  # <Issue>: _write_atomic overwrites the file; previous events are lost.

            # Write to audit file
            prev_event_id = "0" * 64
            if os.path.exists(self.audit_file):
                with open(self.audit_file, 'rb') as f:
                    try:
                        last_line = f.readlines()[-1].strip()
                        prev_event_id = last_line.split()[1].decode()
                    except IndexError:
                        pass  # If audit file is empty

            # <Issue>: Chain formula should be sha256(prev_event_id || record_json), but code uses json.dumps(record) again (may differ) and mismatches verify().
            chain_hash = hashlib.sha256((prev_event_id + json.dumps(record)).encode()).digest()
            self._write_atomic(self.audit_file, f"{base64.urlsafe_b64encode(chain_hash).decode()} {event_id}\n")

            self._rotate_events_if_needed()

        return event_id

    def _write_atomic(self, file_path: str, content: str) -> None:
        """Write content to a file atomically."""
        # <Issue>: This function writes only *content*, discarding all existing lines instead of appending.
        with tempfile.NamedTemporaryFile('w', delete=False, dir=self.root_dir) as temp_file:
            temp_file.write(content)
            temp_path = temp_file.name
        os.replace(temp_path, file_path)

    def _rotate_events_if_needed(self) -> None:
        """Rotate the events file if its size exceeds the maximum limit."""
        if os.path.getsize(self.events_file) > MAX_EVENTS_FILE_SIZE:
            timestamp = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
            new_name = f'events-{timestamp}.jsonl'
            os.rename(self.events_file, os.path.join(self.root_dir, new_name))  # <Issue>: Does not create a fresh empty events.jsonl after rotation.

    def verify(self) -> bool:
        """Verify the integrity of the audit log."""
        if not os.path.exists(self.audit_file):
            return True

        try:
            with open(self.audit_file, 'r') as f:
                prev_event_id = "0" * 64
                for line in f:
                    chain_hash_encoded, event_id = line.strip().split()
                    expected_chain_hash = hashlib.sha256((prev_event_id + event_id).encode()).digest()  # <Issue>: Should hash prev_event_id || event_json, not event_id.
                    if base64.urlsafe_b64encode(expected_chain_hash).decode() != chain_hash_encoded:
                        return False
                    prev_event_id = event_id
        except Exception:
            return False  # <Issue>: Broad except hides bugs; should log or re‑raise specific errors.

        return True


class WebhookRequestHandler(BaseHTTPRequestHandler):
    def __init__(self, *args, root_dir=None, logger: EventLogger = None, **kwargs):
        self.root_dir = root_dir
        self.logger = logger
        super().__init__(*args, **kwargs)

    def do_POST(self):
        if self.path != '/webhook':
            self.send_error(404, 'Not Found', '{"error":"Path not found"}')  # <Issue>: send_error returns HTML by default; custom body may be ignored.
            return

        content_length = self.headers.get('Content-Length')
        if content_length is None:
            self.send_error(411, 'Length Required', '{"error":"Content-Length header is required"}')
            return

        try:
            content_length = int(content_length)
            if content_length > MAX_CONTENT_LENGTH:
                self.send_error(413, 'Payload Too Large', '{"error":"Payload too large"}')
                return
        except ValueError:
            self.send_error(411, 'Length Required', '{"error":"Content-Length header is invalid"}')
            return

        content_type = self.headers.get('Content-Type')
        if content_type != 'application/json':  # <Issue>: Rejects valid 'application/json; charset=utf-8'.
            self.send_error(415, 'Unsupported Media Type', '{"error":"Content-Type must be application/json"}')
            return

        try:
            post_data = self.rfile.read(content_length)
            if b'\r' in post_data or b'\n' in post_data:
                raise ValueError("Control character in body")
            payload = json.loads(post_data.decode('utf-8'))
        except (json.JSONDecodeError, UnicodeDecodeError, ValueError):
            self.send_error(400, 'Bad Request', '{"error":"Invalid JSON or UTF-8 encoding"}')
            return

        try:
            event_id = self.logger.append(payload)
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'status': 'ok', 'id': event_id}).encode('utf-8'))
        except (TypeError, ValueError) as e:
            self.send_error(422, 'Unprocessable Entity', f'{{"error":"{str(e)}"}}')

    def log_message(self, format, *args):
        # Override to prevent default stdout logging
        return  # <Issue>: Good – suppresses noisy server logs.


def run_server(root_dir, host='0.0.0.0', port=8000):
    logger = EventLogger(root_dir)

    def handler(*args, **kwargs):
        WebhookRequestHandler(*args, root_dir=root_dir, logger=logger, **kwargs)

    httpd = HTTPServer((host, port), handler)
    print(f"Starting server on {host}:{port}")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    finally:
        httpd.server_close()
        print("Server stopped.")


def cli():
    import argparse

    parser = argparse.ArgumentParser(description="Webhook Event Logger")
    subparsers = parser.add_subparsers(dest='command', help='sub-command help')

    serve_parser = subparsers.add_parser('serve', help='Start the HTTP server')
    serve_parser.add_argument('--root', required=True, help='Root directory for event logs')
    serve_parser.add_argument('--host', default='0.0.0.0', help='Host to run the server on')
    serve_parser.add_argument('--port', type=int, default=8000, help='Port to run the server on')

    verify_parser = subparsers.add_parser('verify', help='Verify the audit log')
    verify_parser.add_argument('--root', required=True, help='Root directory for event logs')

    args = parser.parse_args()

    if args.command == 'serve':
        run_server(root_dir=args.root, host=args.host, port=args.port)
    elif args.command == 'verify':
        logger = EventLogger(args.root)
        result = logger.verify()
        if result:
            print("Audit log OK")
        else:
            print("Audit log tampered")
        exit(0 if result else 1)


if __name__ == '__main__':
    cli()

```